[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?urlpath=lab/tree/examples/PyMPDATA_examples/advection_diffusion_2d/advection-diffusion-2d.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/examples/blob/main/examples/PyMPDATA_examples/advection_diffusion_2d/advection-diffusion-2d.ipynb)

In [ ]:
from open_atmos_jupyter_utils import show_plot

TOC:
- run a basic constant-coefficient advection-diffusion simulation with Trixi.jl
- run analogous simulation with PyMPDATA
- compare output

## common settings

In [ ]:
SETUP = {
    "nx": 32,
    "ny": 32,
    "ux": 0.5,
    "uy": 0.25,
    "dt": 0.025,
    "tmax": 1.0
}

import json
with open('setup1.json', 'w', encoding='UTF-8') as f:
    json.dump(SETUP, f)

## Trixi.jl

In [ ]:
%%writefile trixi.jl
import Pkg
Pkg.add(["JSON", "Trixi", "OrdinaryDiffEq"])
using JSON
using Trixi
using OrdinaryDiffEq


setup = JSON.parsefile("setup.json")
advection_velocity = (setup["Cx"], setup["Cy"])
equations = LinearScalarAdvectionEquation2D(advection_velocity)

solver = DGSEM(polydeg = 3)

cells_per_dimension = (setup["nx"], setup["ny"])
coordinates_min = (-1.0, -1.0)
coordinates_max = ( 1.0,  1.0)

mesh = StructuredMesh(cells_per_dimension, coordinates_min, coordinates_max)
semi = SemidiscretizationHyperbolic(mesh, equations, initial_condition_convergence_test, solver)

tspan = (0.0, setup["tmax"])
ode = semidiscretize(semi, tspan);

summary_callback = SummaryCallback()
save_solution = SaveSolutionCallback(interval=100)

stepsize_callback = StepsizeCallback(cfl = 1.6)

callbacks = CallbackSet(summary_callback, save_solution, stepsize_callback)

time_int_tol = 1e-6
sol = solve(ode, CarpenterKennedy2N54();
            abstol = time_int_tol,
            reltol = time_int_tol,
            dt = setup["dt"],
            ode_default_options()..., callback = callbacks);

summary_callback()

In [ ]:
%%bash
julia trixi.jl 2>&1

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

with h5py.File('C:/Users/Pawel/out/solution_000031.h5', 'r') as f:
    # average every 16 points to reduce the number of points
    temp = np.array([np.mean(x) for x in f['variables_1'][:].reshape(-1, 16)])
    plt.imshow(temp[:].reshape(SETUP['ny'], SETUP['nx']).transpose())
    plt.show()

## PyMPDATA

In [ ]:
import os
import sympy as sp
from IPython.display import display
from ipywidgets import FloatProgress
from PyMPDATA import Solver, ScalarField, VectorField, Stepper, Options
from PyMPDATA.boundary_conditions import Periodic

In [ ]:
steps_per_iter = 1
opt = Options(n_iters=3, non_zero_mu_coeff=True, infinite_gauge=True, nonoscillatory=True)

In [ ]:
x0 = -1.
y0 = -1.
boundary_conditions = (Periodic(), Periodic())

In [ ]:
mu = 0.05
min_x, min_y = -1, -1
max_x, max_y = 1, 1
dx = (max_x - min_x) / SETUP['nx']
dy = (max_y - min_y) / SETUP['ny']
nt = int(SETUP['tmax'] / SETUP['dt'])
Cx = SETUP['ux'] * SETUP['dt'] / dx
Cy = SETUP['uy'] * SETUP['dt'] / dy
# Cx = 0
# Cy = 0
# solution_symbolic = sp.sympify("sin(pi*(x+y))*exp(-mu*pi**2*t) + 1", rational=True)
solution_symbolic = sp.sympify("exp(-100*(x**2 + y**2)) + 1", rational=True) #exp(-100 * (x[1]^2 + x[2]^2)))
solution = solution_symbolic.subs({"mu": mu})

In [ ]:
def init_conditions(x, y):
    return solution_symbolic.subs({"t":0, "x": x, "y": y}).evalf()

In [ ]:
z = np.array(
    [
        # [
            init_conditions(x, y) for x in np.linspace(min_x, max_x, SETUP['nx'])
        # ]
        for y in np.linspace(min_y, max_y, SETUP['ny'])
    ],
    dtype=float
).reshape((SETUP['nx'], SETUP['ny']))

In [ ]:
advectee = ScalarField(data=z, halo=opt.n_halo, boundary_conditions=boundary_conditions)

In [ ]:
field_x = np.full((SETUP['nx']+1, SETUP['ny']), Cx, dtype=opt.dtype)
field_y = np.full((SETUP['nx'], SETUP['ny']+1), Cy, dtype=opt.dtype)
advector = VectorField(
    data=(field_x, field_y),
    halo=opt.n_halo,
    boundary_conditions=(boundary_conditions[0], Periodic())
)

In [ ]:
stepper = Stepper(options=opt, n_dims=2)

In [ ]:
# create a solver
solver = Solver(stepper=stepper, advector=advector, advectee=advectee)

In [ ]:
plt.imshow(solver.advectee.get().copy(), cmap='viridis')
plt.colorbar()
show_plot()

In [ ]:
progbar = FloatProgress(value=0, min=0, max=1)
display(progbar)

# states_history = [solver.advectee.get().copy()]
# n_steps = 10
# nt = int(SETUP['tmax'] / SETUP['dt'])
# n_range = int(SETUP['tmax'] / SETUP['dt']) // n_steps
# for i in range(n_range):
#     solver.advance(n_steps=n_steps, mu_coeff=(mu, mu))
#     states_history.append(solver.advectee.get().copy())
#     progbar.value = (i + 1.) / n_range
    
states_history = [solver.advectee.get().copy()]
for i in range(10*nt):
    solver.advance(n_steps=1, mu_coeff=(mu, mu))
    states_history.append(solver.advectee.get().copy())
    progbar.value = (i + 1.) / (10*nt)

In [ ]:
plt.imshow(states_history[-1], cmap='viridis')  
plt.colorbar()
plt.show()
# show_plot()

solution_symbolic_final = sp.sympify("sin(pi*(x-cx*dx*nt + y-cy*dx*nt)) + 1", rational=True)
def final_conditions(x, y):
    return solution_symbolic_final.subs({"cx": Cx, "cy": Cy, "t":SETUP['tmax'], "dx": dx, "dy": dy, "nt": nt, "x": x, "y": y}).evalf()

z_final = np.array(
    [
        # [
            final_conditions(x, y) for x in np.linspace(min_x, max_x, SETUP['nx'])
        # ]
        for y in np.linspace(min_y, max_y, SETUP['ny'])
    ],
    dtype=float
).reshape((SETUP['nx'], SETUP['ny']))
plt.imshow(z_final, cmap='viridis')
plt.colorbar()
plt.show()
with h5py.File('C:/Users/Pawel/out/solution_000031.h5', 'r') as f:
    # average every 16 points to reduce the number of points
    temp = np.array([np.mean(x) for x in f['variables_1'][:].reshape(-1, 16)])
    plt.imshow(temp[:].reshape(SETUP['ny'], SETUP['nx']).transpose())
    plt.show()

In [ ]:
trixi_result = temp[:].reshape(SETUP['ny'], SETUP['nx']).transpose()

# Calculate rmse between Trixi and PyMPDATA output
rmse = np.sqrt(np.mean((states_history[-1] - trixi_result) ** 2))
print(f"RMSE: {rmse:.4f}")

# Calculate min-max difference between Trixi and PyMPDATA output
min_max_diff = np.max(np.abs(states_history[-1] - trixi_result))
print(f"Min-max difference: {min_max_diff:.4f}")

In [ ]:
with h5py.File('out/solution_000000.h5', 'r') as f:
    trixi_init = np.array([np.mean(x) for x in f['variables_1'][:].reshape(-1, 16)])
    trixi_init = trixi_init[:].reshape(SETUP['ny'], SETUP['nx']).transpose()

In [ ]:
assert np.abs(np.sum(states_history[-1]) - np.sum(states_history[0])) < 1e-6
assert np.abs(np.sum(trixi_result) - np.sum(trixi_init)) < 1e-6

In [ ]:
solution_symbolic_final = sp.sympify("sin(pi*(x-cx*t + y-cy*t)) + 1", rational=True)
def final_conditions(x, y):
    return solution_symbolic_final.subs({"cx": SETUP['Cx'], "cy": SETUP['Cy'], "t":SETUP['tmax'], "x": x, "y": y}).evalf()

z_final = np.array(
    [
        # [
            final_conditions(x, y) for x in np.linspace(min_x, max_x, SETUP['nx'])
        # ]
        for y in np.linspace(min_y, max_y, SETUP['ny'])
    ],
    dtype=float
).reshape((SETUP['nx'], SETUP['ny']))

plt.imshow(z_final, cmap='viridis')
plt.colorbar()
show_plot()

In [ ]:
# Calculate rmse between analytical solution, Trixi and PyMPDATA output
rmse = np.sqrt(np.mean((states_history[-1] - z_final) ** 2))
print(f"PyMPDATA - RMSE: {rmse:.4f}")

rmse = np.sqrt(np.mean((trixi_result - z_final) ** 2))
print(f"Trixi - RMSE: {rmse:.4f}")

In [ ]:
# Calculate min-max difference between analytical solution, Trixi and PyMPDATA output
min_max_diff = np.max(np.abs(states_history[-1] - z_final))
print(f"PyMPDATA - Min-max difference: {min_max_diff:.4f}")

min_max_diff = np.max(np.abs(trixi_result - z_final))
print(f"Trixi - Min-max difference: {min_max_diff:.4f}")

In [ ]:
os.makedirs("animation", exist_ok=True)
for i, state in enumerate(states_history):
    state = np.flipud(state)
    plt.imshow(state, cmap='viridis')
    plt.axis('off')
    plt.tight_layout()
    plt.colorbar()
    plt.savefig(f"animation2/frame_{i:03d}.png")
    plt.close()

In [ ]:
def merge_images_into_gif(image_folder, gif_name, duration=0.1):
    import imageio
    with imageio.get_writer(gif_name, mode='I', duration=duration) as writer:
        for filename in sorted(os.listdir(image_folder)):
            image = imageio.imread(os.path.join(image_folder, filename))
            writer.append_data(image)

In [ ]:
merge_images_into_gif("animation2", "advection_diffusion.gif", duration=0.01)